# PBL03 加工内容の図面解析による自動見積もり

## 1. ライブラリimport

In [3]:
!pip install -r requirements.txt

In [4]:
import os
import sys
import random
import warnings
import json
import tqdm
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import detection_pipeline

import xml.etree.ElementTree as ET

# TensorFlowのimport
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import ModelCheckpoint

from inference.utils import *
from inference.Evaluator import *

## 2. パス設定

In [ ]:
# カレントディレクトリの取得
root_path = os.getcwd()
root_path = "/content/drive/MyDrive/SIGNATE/AI-Quest/"
root_path

In [6]:
# 各種パスの設定
yolo_attributes_path = root_path + '/yolo_attributes/'
data_attributes_path = root_path + '/data_attributes/'
dataset_path = root_path + '/datasets/' 

if not os.path.exists(data_attributes_path):os.makedirs(data_attributes_path)

## 3. パラメータ設定

In [7]:
# 物体検出アルゴリズムに関する設定
INPUT_SHAPE = (320, 320)  # 物体検出アルゴリズムへの入力画像の幅と高さ。32の倍数でなければならない
BATCH_SIZE = 4 # バッチサイズ
#NUM_EPOCHS = 3 # エポック数
NUM_EPOCHS = 300 # エポック数_全然学習できてなさそうなので増やす
LR = 1e-4 # 学習率

In [8]:
# ランダムシードの固定
seed_value = 12

# 乱数生成器の初期化
random.seed(seed_value)

# https://teratail.com/questions/291319
os.environ['PYTHONHASHSEED'] = str(seed_value)

# numpy、およびTensorFlowの乱数シードを固定
# https://qiita.com/bee2/items/08eab7a899c9ff56eb35
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

## 4. アノテーションデータ（xmlファイル）のtxtファイルへの変換

In [9]:
# 切削穴の種類
classes = ["circle", "round_rectangle", "rectangle", "triangle"]

In [10]:
# xmlファイル：HTMLファイルの親戚。独自に決めたタグ名を使用できるのが大きな違い
# https://www.soubun.com/journal/xml%E3%81%A8%E3%81%AF%EF%BC%9F-html%E3%81%A8%E3%81%AE%E9%81%95%E3%81%84%E3%81%8B%E3%82%89%E3%83%A1%E3%83%AA%E3%83%83%E3%83%88%E3%80%81pdf%E3%81%8B%E3%82%89%E3%81%AE%E5%A4%89%E6%8F%9B%E6%96%B9%E6%B3%95/

# xmlファイルからtxtファイルへの変換用関数
def convert_annotation(input_file, output_file):
    in_file = open(input_file)
    # xmlファイルを解析
    tree = ET.parse(in_file)
    # 解析結果を取得
    root = tree.getroot()

    # xmlを要素名（今回は"object"）を指定して取り出し
    # 以下の処理を穴（=object）の数だけループ
    for obj in root.iter('object'):
        # objectから"name"を探してその中のデータ取得
        # 今回はnameの中に切削穴の種類が書き込まれてる
        # "name"に挟まれた情報（=切削穴の種類）を取得
        cls = obj.find('name').text
        # 取得した切削穴種類：clsが、最初に指定した4つの穴種類：classesの中にない場合はcontinue（=以降の処理はやらずforの先頭に戻る）
        if cls not in classes:
            continue
        # classesリストのindex取得
        cls_id = classes.index(cls)
        # objectから"bndbox"を探してその中のデータを取得
        # bndboxの中には穴の位置情報？が書き込まれてる
        # bndbox内の情報にアクセス
        xmlbox = obj.find('bndbox')
        # bはタプル。xmin, ymin, xmax, ymaxを格納する
        b = (int(float(xmlbox.find('xmin').text)), int(float(xmlbox.find('ymin').text)), int(float(xmlbox.find('xmax').text)), int(float(xmlbox.find('ymax').text)))
        # 指定したファイル（output_file）の中に、位置情報と切削穴種類を書き込む
        output_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))
        
# 各xmlファイルを1つのtxtファイルに集約する関数
def convert_xml_to_txt(data_type: str):
    # ./datasets/ 直下にあるディレクトリパスを取得（今回はtrain, valid, test）
    # パスを取得するディレクトリ名（=data_type）は本モジュールの引数で指定
    ann_paths = dataset_path + f'{data_type}/'
    # 上で指定したディレクトリ（=ann_paths）の中にあるファイル、ディレクトリをリストに格納
    ann_paths_list = os.listdir(ann_paths)
    # ./data_attributes_path/instances_{data_type}.txt のテキストファイルを作成、open
    # 今回は書き込み用でopen（mode="w"）
    output_file = open(data_attributes_path + f'instances_{data_type}.txt', 'w')
    
    # ann_pathsの中にあるファイル、ディレクトリの数だけループ。今回はjpegとxmlファイルだけが入ってる想定
    # tqdm()：処理中にプログレスバーを表示させる
    # https://vector-ium.com/python-tqdm/
    for ann in tqdm.tqdm(ann_paths_list):

        # ann_paths_listの要素（=ann）を"."でsplitしたときの末尾（=拡張子）がxmlのものだけ処理
        if ann.split('.')[-1] == 'xml':
            # xmlファイルへのパス取得
            ann = ann_paths + ann
            # jpegファイルへのパス取得
            # xml→jpegと置換することで取得。"annot_mod"はよくわからん
            image_path = ann.replace('xml','jpeg').replace('annot_mod', 'image')
            # output_file（=上で作ったinstances_{data_type}.txt）にjpegファイルへのパスをwrite
            output_file.write(image_path)
            # さらにoutput_fileにannで指定したxmlファイル内の位置情報と切削穴種類をwrite
            convert_annotation(ann, output_file)
            # 改行も追加
            output_file.write('\n')

    output_file.close()

In [11]:
# train、valid、testフォルダ内の各xmlファイルを1つのtxtファイルに変換
# train, valid, testフォルダに上の処理をそれぞれ実施
convert_xml_to_txt(data_type='train')
convert_xml_to_txt(data_type='valid')
convert_xml_to_txt(data_type='test')

100%|██████████| 190/190 [00:00<00:00, 134455.50it/s]


In [12]:
# 各xmlファイルをtxtファイルに変換する関数
def create_the_ground_truth(data_type: str):

    # 辞書の内包表記{no : i.strip(" ")}
    # class_label.csvの1行目（しかない）をlistにしてから辞書に変換
    # no ： 0〜3（key）
    # i ： circle, round_rectangle, rectangle, triangle（value）
    # iは両端にスペースあったら削除
    # ↓こんな辞書ができる
    # {0: 'circle', 1: 'round_rectangle', 2: 'rectangle', 3: 'triangle'}
    label = {no:i.strip(' ') for no,i in enumerate(pd.read_csv(yolo_attributes_path + 'class_label.csv', 
                                                           header=None).values.tolist()[0])}

    # ./data_attributes/ 直下のファイルやらディレクトリのパス取得（makeはしてない）
    # instances_{data_type}.txt　：　テキストファイル
    # groundtruths_{data_type}　：　ディレクトリ
    ANNOT_DIR = data_attributes_path + f'instances_{data_type}.txt'
    SAVE_DIR = data_attributes_path + f'groundtruths_{data_type}'

    # groundtruths_{data_type}がない場合は作る
    if not os.path.exists(SAVE_DIR): os.makedirs(SAVE_DIR)

    # filesにinstances_{data_type}.txt内の情報（=位置情報と切削穴種類）をread
    # 改行で区切ってるので1枚の図面情報ごとにreadされてる
    files = open(ANNOT_DIR,"r").read().split('\n')
    # 情報なかったらfilesにはreadしない
    files = [file for file in files if file is not '']
    
    # j ：　files内の改行で区切られた範囲ごと（=図面ごと）の情報
    for j in tqdm.tqdm(files):
        # jの中から”　”ごと（=切削穴ごと）に区切ってループ
        # ↑convert_annotationモジュールで切削穴ごとに”　”で区切って書き込んでる
        # ただし” ”で区切った1つ目の要素は図面のパス情報なので以下のループで処理を分けてる
        for no, i in enumerate(j.split(' ')):
            # 1つ目の要素（=図面のパス情報）だけ以下の処理
            if no == 0:
                # https://note.nkmk.me/python-os-basename-dirname-split-splitext/
                # iから拡張子ありのファイル名部分の文字列を返す
                # ここでは”01_01.jpeg”とかの形で返ってくる
                gtname = os.path.basename(i)
                # 上で返ってきたファイル名の拡張子なしver.を返す
                # ここでは”01_01”とか
                gtname = os.path.splitext(gtname)[0]
                # open()：第1引数に指定したパスが示すファイルオブジェクトが開かれる
                # join()：引数のパスを結合
                # SAVE_DIR（=.groundtruths_{data_type}/）の直下に{gtnama}.txtファイルを書き込み用としてopen
                list_file = open(os.path.join(SAVE_DIR, f'{gtname}.txt'), 'w')
            # 2つ目以降（=切削穴の情報）は以下の処理
            else:
                # 位置情報、クラス情報は”,”で区切られてる。”,”で区切ってlist化
                elm = i.split(',')
                # join()　：　文字列のlistを1つの文字列に連結
                # https://note.nkmk.me/python-string-concat/
                # 上で作ったtxtファイルに以下の表記でwrite
                # 例・・・　rectangle　100 200 300 400
                list_file.write(label[int(elm[-1])] + ' ' + ' '.join(elm[:-1]))
                # 改行追加
                list_file.write('\n')

        list_file.close()

In [13]:
# train、valid、testフォルダ内の各xmlファイルをtxtファイルに変換
create_the_ground_truth(data_type='train')
create_the_ground_truth(data_type='valid')
create_the_ground_truth(data_type='test')

100%|██████████| 100/100 [00:53<00:00,  1.63it/s]
0it [00:00, ?it/s]


## 5. 学習・検証データの読み込み

In [14]:
# 各種学習に必要なデータのパスを設定

# 図面のパス情報、穴の位置情報、穴のクラスが書かれてるtxtファイル
train_annotation_path = data_attributes_path + 'instances_train.txt'
valid_annotation_path = data_attributes_path + 'instances_valid.txt'
# Yolo用のアンカー定義ファイル
# アンカー：物体検出する領域について、任意の数のアスペクト比を与えたもの（？）
# 検出領域の形状を複数与えてる？
anchors_path = yolo_attributes_path + 'yolo_anchors.txt'
# 4種のクラスが書いてあるファイル
classes_path = yolo_attributes_path + 'classes_4.txt'
# ./result/weightsへのパス
weight_dir = root_path + '/results/weights/'
# 上のパスがない場合は作る
if not os.path.exists(weight_dir):os.makedirs(weight_dir)

In [15]:
# 学習データの読み込み

# readlines()　：　ファイル全体を行ごとに分割したlistとして取得
# 図面ごとの図面のパス情報、穴の位置情報、穴のクラスをlist化
# ⇨データ数（=図面枚数）分のサイズを持つlistになる
with open(train_annotation_path) as f:
    train_lines = f.readlines()

with open(valid_annotation_path) as f:
    valid_lines = f.readlines()

# 上で作ったlistの要素をシャッフル
# ちなみにrandom_seed値は最初の方で固定している
np.random.shuffle(train_lines)
np.random.shuffle(valid_lines)

# listのサイズ（=データ数）取得
num_train = len(train_lines)
num_valid = len(valid_lines)

print('Total train:', num_train)
print('Total valid:', num_valid)

Total train: 200
Total valid: 100


## 6. 物体検出アルゴリズム Yolo v3 の各種設定

In [16]:
# 学習済みモデルのweightのパスを設定
# ちなみに".h5"はHDF5ファイル。中身はツリー構造のデータらしい
weight_path = yolo_attributes_path + 'yolo3_weights.h5'

In [17]:
# Yolo v3の設定

# 別で用意した"detection_pipeline.py"内で定義されているget_anchorsモジュールを使う
# 指定したファイルからanchor情報をloadしてlist化する
anchors = detection_pipeline.get_anchors(anchors_path)
# 同じくget_classesモジュールを使う
# 指定したファイルからクラス情報をloadしてlist化する
class_names = detection_pipeline.get_classes(classes_path)

# クラス数を取得
num_classes = len(class_names)

# 学習モデルの作成
# create_modelモジュールを使って
model = detection_pipeline.create_model(
            # 物体検出アルゴリズムへの入力画像の幅と高さ
            # 今回は(320, 320)。32の倍数じゃないとダメらしい
            INPUT_SHAPE,
            # "yolo_anchors.txt"内にあるanchor情報を指定
            anchors,
            # 今回のクラス数を指定
            num_classes,
            # freezeする（=学習対象から外す）レイヤーの数？を指定
            # （create_modelモジュールのコードを見る限り、おそらく）1or2を指定する
            # 1　：　185/252を固定　　
            # 2　：　249/252を固定
            # ⇨今回は多めに固定してる
            freeze_body=2,
            # 今回は学習済みモデルを使う
            # create_modelモジュール内で"load_pretrained = True"となってるので、ここで指定した学習済みの重み情報を読み込んでくれる
            weights_path=weight_path)

Create YOLOv3 model with 9 anchors and 4 classes.
Load weights /content/drive/MyDrive/SIGNATE/AI-Quest/Term1/Phase2_AI課題/aiquest_pbl03_sample/yolo_attributes/yolo3_weights.h5.
Freeze the first 249 layers of total 252 layers.
now it is in layer 0 Tensor("yolo_loss/strided_slice_5:0", shape=(None, 10, 10, 3, 1), dtype=float32)
now it is in layer 1 Tensor("yolo_loss/strided_slice_31:0", shape=(None, 20, 20, 3, 1), dtype=float32)
now it is in layer 2 Tensor("yolo_loss/strided_slice_57:0", shape=(None, 40, 40, 3, 1), dtype=float32)


## 7. モデルの学習

In [18]:
# モデルの名前を定義
saved_model_name = 'baseline_model'

In [19]:
# best epoch時のweightを保存
# 今回は"best_baseline_model.h5"に保存
mc = ModelCheckpoint(weight_dir + f'best_{saved_model_name}.h5',
                     monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [20]:
# モデルの学習

# 上で作成したモデルの全レイヤー？を学習可能な状態（=trainanble）にする
for layer in model.layers:
    layer.trainable = True
    
# recompile to apply the change
# 変更を反映するためにrecompile。。。らしい

# optimizer ：　最適化アルゴリズムの種類。今回はAdam。()内は学習率
# ⇨ちゃんとtensorflow.keras.optimizersクラスからimportしてる
# （from tensorflow.keras.optimizers import Adam）

# loss ：　損失関数の種類。今回は ./yolo3/model.py 内で定義しているyolo_lossモジュール（=実際と予測のピクセルのズレ？）を使ってる
# ⇨今回はカスタムした損失関数使ってるが、既存の組み込み済みのものを使うのも可能
# https://atmarkit.itmedia.co.jp/ait/articles/2003/23/news024.html
# https://qiita.com/nagataaaas/items/531b1fc5ce42a791c7df

model.compile(optimizer=Adam(lr=LR), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) 
print('Unfreeze all of the layers.')

print('Train on {} samples, valid on {} samples, with batch size {}.'.format(num_train, num_valid, BATCH_SIZE))

# バッチサイズとエポック数
# 入力はバッチサイズ分の列次元を持つ行列になる
# https://python.atelierkobato.com/batch/
# https://note.com/yeahyeahoh/n/nb979ed7e9cb8
# https://note.com/yeahyeahoh/n/n98fdacb4828f

# fit_generatorメソッドについて
# fitメソッドとは異なり、データを生成するジェネレータとステップ数を与える必要がある
# ⇨今回はそのジェネレータがdata_generator_wrapper()
# https://hironsan.hatenablog.com/entry/2017/09/09/130608

#　wrapperについて
# https://yu-nix.com/blog/2020/12/1/programming-wrapper/

# ↓trainデータとvalidデータのそれぞれにdata_generator()を使ってる
model.fit_generator(detection_pipeline.data_generator_wrapper(
                        train_lines, 
                        BATCH_SIZE, 
                        INPUT_SHAPE, 
                        anchors, 
                        num_classes),
                    steps_per_epoch=max(1, num_train//BATCH_SIZE), 
                    validation_data=detection_pipeline.data_generator_wrapper(
                        valid_lines, 
                        BATCH_SIZE, 
                        INPUT_SHAPE, 
                        anchors, 
                        num_classes), 
                    validation_steps=max(1, num_valid//BATCH_SIZE), 
                    epochs=NUM_EPOCHS,
                    callbacks=[mc])

# final epoch時のweightを保存
model.save_weights(weight_dir + f'final_{saved_model_name}.h5')

Unfreeze all of the layers.
Train on 200 samples, valid on 100 samples, with batch size 4.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/300
now it is in layer 0 Tensor("functional_5/yolo_loss/strided_slice_5:0", shape=(None, 10, 10, 3, 1), dtype=float32)
now it is in layer 1 Tensor("functional_5/yolo_loss/strided_slice_31:0", shape=(None, 20, 20, 3, 1), dtype=float32)
now it is in layer 2 Tensor("functional_5/yolo_loss/strided_slice_57:0", shape=(None, 40, 40, 3, 1), dtype=float32)
now it is in layer 0 Tensor("functional_5/yolo_loss/strided_slice_5:0", shape=(None, 10, 10, 3, 1), dtype=float32)
now it is in layer 1 Tensor("functional_5/yolo_loss/strided_slice_31:0", shape=(None, 20, 20, 3, 1), dtype=float32)
now it is in layer 2 Tensor("functional_5/yolo_loss/strided_slice_57:0", shape=(None, 40, 40, 3, 1), dtype=float32)
50/50 [==============================] - ETA: 0s - loss: 2740.9053now it is in layer 0 Tensor("functional_5/yolo_loss/strided_s

## 8. モデルによる切削穴の検出

In [21]:
# メモリ使用量削除のためセッションのクリア
tf.compat.v1.disable_eager_execution()
tf.keras.backend.clear_session()

In [ ]:
# 切削穴検出（推論）のための各種yolo v3の設定

# 「７．モデルの学習」で学習したfinal epoch時のweightを指定
model_weights = os.path.join(weight_dir, f'final_{saved_model_name}.h5')
# 上の学習で得たweightを持ったモデルをload
yolo_model = detection_pipeline.load_model(model_weights, anchors_path, classes_path, INPUT_SHAPE)

In [23]:
# 切削穴検出用図面のパスと検出結果の格納先の設定
saved_prediction_dir = root_path + '/results/inferences/'
# create save_path if not exist
# 推論結果保存用のディレクトリ（=inferences）がない場合は作る
if not os.path.exists(saved_prediction_dir):os.makedirs(saved_prediction_dir)

#train, valid, testそれぞれのdataset（.jpeg & .xml）と推論結果保存先へのパス取得
image_dir_train = dataset_path + 'train/'
save_path_train = saved_prediction_dir + 'train/'    

image_dir_valid = dataset_path + 'valid/'
save_path_valid = saved_prediction_dir + 'valid/'

image_dir_test = dataset_path + 'test/'
save_path_test = saved_prediction_dir + 'test/'

# create save_path if not exist
if not os.path.exists(save_path_train): os.makedirs(save_path_train)

# create save_path if not exist
if not os.path.exists(save_path_valid): os.makedirs(save_path_valid)

# create save_path if not exist
if not os.path.exists(save_path_test): os.makedirs(save_path_test)

In [24]:
# 切削穴検出と検出結果の保存
def inference_and_save_result(image_dir: str, save_path: str):
    
    for file in os.listdir(image_dir):
        # 画像ファイル（=推論対象）じゃなかったらループ抜ける
        # ⇨今回はxmlファイルがスキップ
        if not file.endswith(('.jpg', '.jpeg', '.png')):
            continue

        # 対象図面へのパス取得
        image_path = os.path.join(image_dir, file)

        # load image
        image = load_img(image_path)
        
        # make prediction
        # detect_image()　：　./yolo3/yolo.py 内のYOLOクラス内で定義されてる
        # "yolo_model"はYOLOクラスのインスタンス（のはず）
        # ↓detect_imageモジュール内では"self.sess.run()"でpredictしてる（っぽい）
        # https://qiita.com/cvusk/items/a2a0a11815de491cf3e5
        img_data, output = yolo_model.detect_image(image)

        # get image name
        # 図面ファイルから拡張子切り離し
        image_name = file.split('.')[0]
        # 推論結果をtxtファイルに出力
        # そのtxtファイルをsave_pathの場所に保存
        detection_pipeline.save_prediction(output, image_name, classes_path, save_path)

# 各ディレクトリにある図面ファイルで検出実施
print('TRAIN')
inference_and_save_result(image_dir_train, save_path_train)

print('VALID')
inference_and_save_result(image_dir_valid, save_path_valid)

print('TEST')
inference_and_save_result(image_dir_test, save_path_test)

TRAIN
Found 20 boxes for img
Found 6 boxes for img
Found 20 boxes for img
Found 4 boxes for img
Found 3 boxes for img
Found 0 boxes for img
Found 5 boxes for img
Found 5 boxes for img
Found 4 boxes for img
Found 5 boxes for img
Found 4 boxes for img
Found 2 boxes for img
Found 4 boxes for img
Found 5 boxes for img
Found 3 boxes for img
Found 6 boxes for img
Found 2 boxes for img
Found 9 boxes for img
Found 5 boxes for img
Found 9 boxes for img
Found 3 boxes for img
Found 4 boxes for img
Found 3 boxes for img
Found 3 boxes for img
Found 2 boxes for img
Found 1 boxes for img
Found 9 boxes for img
Found 1 boxes for img
Found 9 boxes for img
Found 7 boxes for img
Found 7 boxes for img
Found 4 boxes for img
Found 6 boxes for img
Found 4 boxes for img
Found 2 boxes for img
Found 7 boxes for img
Found 8 boxes for img
Found 1 boxes for img
Found 2 boxes for img
Found 7 boxes for img
Found 7 boxes for img
Found 9 boxes for img
Found 1 boxes for img
Found 10 boxes for img
Found 4 boxes for img
F

In [25]:
# アノテーションデータと検出結果のパス設定
gt_dir_train = data_attributes_path + 'groundtruths_train/'
gt_dir_valid = data_attributes_path + 'groundtruths_valid/'

In [26]:
# precision、recallの算出と検出結果を可視化した画像の生成・保存 image file
def calculate_score_and_visualize_inferences(image_dir, gt_dir, pred_dir):

    # define the visualization storage and make it
    save_into = os.path.join(pred_dir + 'drawings/')
    if not os.path.exists(save_into): os.makedirs(save_into)

    for file in os.listdir(pred_dir):

        if file.endswith('.txt'):

            # get image name
            image_name = file.split('.')[0]

            # define the final access path for image, ground truth, and predictions
            image_path = os.path.join(image_dir, image_name) + '.jpeg'
            gt_result_path = os.path.join(gt_dir, file)
            pred_result_path = os.path.join(pred_dir, file)

            # get score for each image
            detection_pipeline.get_individual_score(gt_result_path, pred_result_path)

            # visualize the results
            detection_pipeline.visualize_output(image_path, gt_result_path, pred_result_path, save_into, image_name)

print('TRAIN')
calculate_score_and_visualize_inferences(image_dir_train, gt_dir_train, save_path_train)

print('VALID')
calculate_score_and_visualize_inferences(image_dir_valid, gt_dir_valid, save_path_valid)

Output hidden; open in https://colab.research.google.com to view.

## 9. 検出結果のF1 Scoreによる評価

In [27]:
# F1 score算出用関数
def get_f1_score(gt_dir, pred_dir):
    
    boundingboxes = detection_pipeline.get_all_boundingboxes(gt_dir, pred_dir)
    
    evaluator = Evaluator()

    metricsPerClass = evaluator.GetPascalVOCMetrics(boundingboxes, IOUThreshold=0.3)

    f1_scores = []
    
    print(f'Average precision values per class of data:\n')

    for mc in metricsPerClass:
    
        c = mc['class']
        precision = mc['precision']
        recall = mc['recall']
        average_precision = mc['AP']
        ipre = mc['interpolated precision']
        irec = mc['interpolated recall']
        total_gt = mc['total positives']
        tp = mc['total TP']
        fp = mc['total FP']

        # Print AP per class
        print('---')
        print(f'{c}: {round(average_precision,2)}')

        # print precision (self)
        precision_self = tp/(tp+fp)
        print(f'Precision of {c} = {round(precision_self,2)}')

        # print recall (self)
        recall_self = tp/total_gt
        print(f'Recall of {c} = {round(recall_self,2)}')
        
        f1_score_self = (2 * precision_self * recall_self) / (precision_self + recall_self)
        print(f'F1 Score of {c} = { round(f1_score_self,2) }')
        f1_scores.append(f1_score_self)
        
    print('\n')
    print(f'Average F1 Score:', round(sum(f1_scores)/len(f1_scores),2), '\n')

In [28]:
# 学習・検証データに対するF1 score
print('TRAIN')
get_f1_score(gt_dir_train, save_path_train)

print('VALID')
get_f1_score(gt_dir_valid, save_path_valid)

TRAIN
Average precision values per class of data:

---
circle: 0.66
Precision of circle = 0.82
Recall of circle = 0.71
F1 Score of circle = 0.76
---
rectangle: 0.65
Precision of rectangle = 0.83
Recall of rectangle = 0.67
F1 Score of rectangle = 0.74
---
round_rectangle: 0.62
Precision of round_rectangle = 0.92
Recall of round_rectangle = 0.65
F1 Score of round_rectangle = 0.76
---
triangle: 0.59
Precision of triangle = 0.85
Recall of triangle = 0.62
F1 Score of triangle = 0.72


Average F1 Score: 0.74 

VALID
Average precision values per class of data:

---
circle: 0.47
Precision of circle = 0.73
Recall of circle = 0.54
F1 Score of circle = 0.62
---
rectangle: 0.43
Precision of rectangle = 0.65
Recall of rectangle = 0.51
F1 Score of rectangle = 0.57
---
round_rectangle: 0.23
Precision of round_rectangle = 0.38
Recall of round_rectangle = 0.45
F1 Score of round_rectangle = 0.41
---
triangle: 0.06
Precision of triangle = 0.69
Recall of triangle = 0.08
F1 Score of triangle = 0.14


Avera

## 10. 提出ファイルの出力

テストデータに対して切削穴検出を行った結果を提出フォーマットの形式に変換し、出力します。

In [29]:
# 検出結果のtxtファイルを１つの辞書形式のデータに集約
dict_to_json = dict()

for text_file in os.listdir(save_path_test):
        
    if text_file.endswith('.txt'):
        
        data = { 'objects': [], 'confidence_score': [], 'xmin': [], 'ymin': [], 'xmax': [], 'ymax': [] }
        df_data = pd.DataFrame()
        
        f = open(save_path_test + '/' + text_file, 'r') 
        contents = f.readlines()
        
        for string in contents:
            list_of_contents = string.replace('\n', '').split(' ')
            
            data['objects'].append(list_of_contents[0])
            data['confidence_score'].append(float(list_of_contents[1]))
            data['xmin'].append(float(list_of_contents[2]))
            data['ymin'].append(float(list_of_contents[3]))
            data['xmax'].append(float(list_of_contents[4]))
            data['ymax'].append(float(list_of_contents[5]))

        df_data = df_data.from_dict(data)
        df_data_sorted  = df_data.sort_values(by='confidence_score', axis=0, ascending=False)
        
        text_file = text_file.replace('txt', 'jpeg')
        dict_to_json[text_file] = dict()
        
        for data in df_data_sorted.values.tolist():
                        
            if data[0] in dict_to_json[text_file]:
                dict_to_json[text_file][data[0]].append([data[2], data[3], data[4], data[5]])
            else:
                dict_to_json[text_file][data[0]] = [[data[2], data[3], data[4], data[5]]]

In [30]:
# 辞書形式のデータを提出フォーマットであるjsonファイルとして書き出す
with open(save_path_test + '/' + 'sample_submit_210927.json', 'w') as contents: json.dump(dict_to_json, contents)

---